In [ ]:
# PS-7 Password Cracking
# This notebook now uses the modular pipeline

from password_cracker import PasswordCracker, crack_single_file, crack_directory
from pathlib import Path
import config

print('✓ Password cracker module imported')
print(f'\nTo crack files: python pipeline.py crack Mock/')
print(f'Or use PasswordCracker class below.')

In [ ]:
# Example: Crack a single file
# Uncomment to run:

# result = crack_single_file(
#     target_file=Path('Mock/Gc_PS7_Mock_test1.docx'),
#     candidates_file=Path('output/candidates/candidates_combined_ge6.txt'),
#     max_tries=5000
# )
# print(result)

# Or crack entire directory:
# results = crack_directory(
#     target_dir=Path('Mock'),
#     candidates_file=Path('output/candidates/candidates_combined_ge6.txt'),
#     results_file=Path('output/results/crack_results.csv'),
#     max_tries=10000
# )

print('Uncomment code above to crack files.')
print(f'\nAvailable candidates file: output/candidates/candidates_combined_ge6.txt')
print(f'Number of mock files: {len(list(Path("Mock").glob("*")))}')